# 🧪 QEPC v80 – NBA Multi-Season Backtest

This notebook runs a full QEPC pipeline for NBA:

1. Load 10-year NBA logs from `NBA_API_QEPC_Format.csv`.
2. Compute team strengths (ORtg / DRtg / Pace / Volatility).
3. Build a game-level backtest table from logs.
4. Map each game to λ (lambda_home, lambda_away).
5. Run a QEPC multiverse simulation.
6. Evaluate accuracy: win %, total MAE, spread MAE, calibration.

Everything uses the **experimental QEPC core** under `experimental/GTP_REWRITE/qepc_core`.


In [ ]:
from pathlib import Path
import sys

print("=== QEPC Experimental Bootstrap ===")

cwd = Path.cwd()
core_root = None
cur = cwd

# Walk up until we find the qepc_core folder
for _ in range(8):
    if cur.name == "qepc_core":
        core_root = cur
        break
    if cur.parent == cur:
        break
    cur = cur.parent

if core_root is None:
    raise RuntimeError(f"Could not find qepc_core above {cwd}")

core_str = str(core_root)
if core_str not in sys.path:
    sys.path.insert(0, core_str)

# Repo root is 3 levels above qepc_core: .../qepc_project/experimental/GTP_REWRITE/qepc_core
repo_root = core_root.parent.parent.parent
repo_str = str(repo_root)
if repo_str not in sys.path:
    sys.path.append(repo_str)

print("qepc_core root:", core_root)
print("repo root:     ", repo_root)

import qepc
from qepc.config import detect_project_root, QEPCConfig

project_root = detect_project_root()
cfg = QEPCConfig.from_project_root(project_root)

print("project_root:  ", project_root)
print("data/raw:      ", cfg.raw_root)
print("=== Bootstrap OK ===")


In [ ]:
import pandas as pd

from qepc.nba.data_loaders import load_nba_team_logs

logs = load_nba_team_logs(cfg)

print("✅ Loaded NBA logs")
print("Rows:", len(logs))
print("Columns:", len(logs.columns))
print("Date range:", logs["gameDate"].min(), "→", logs["gameDate"].max())
print("Seasons:", logs["Season"].min(), "→", logs["Season"].max())

display(logs.head())


In [ ]:
from qepc.core.strengths import TeamStrengthsConfig, compute_team_strengths

# Configure strengths – tweak later if you like
str_cfg = TeamStrengthsConfig(
    lookback_seasons=3,          # last 3 seasons
    min_games=50,                # min team games required
    recency_half_life_games=60,  # down-weight older games
)

strengths_df = compute_team_strengths(
    game_logs=logs,
    config=str_cfg,
    cutoff_date=None,   # or a Timestamp for historical cut
    verbose=True,
)

print("\nPreview of strengths table:")
display(strengths_df.head(10))

print("\nTotal teams in strengths_df:", len(strengths_df))


In [ ]:
import numpy as np

print("logs shape:", logs.shape)

games = []

for game_id, g in logs.groupby("gameId"):
    g = g.sort_values("home")  # usually 0 then 1

    game_date = g["gameDate"].iloc[0]

    home_row = g[g["home"] == 1]
    away_row = g[g["home"] == 0]

    if len(home_row) == 1 and len(away_row) == 1:
        # Nice, standard case
        h = home_row.iloc[0]
        a = away_row.iloc[0]

        home_team = f"{h['teamCity']} {h['teamName']}".strip()
        away_team = f"{a['teamCity']} {a['teamName']}".strip()

        home_score = int(h["teamScore"])
        away_score = int(a["teamScore"])

    else:
        # Fallback for any weird cases (should be rare)
        h = g.iloc[0]
        home_team = f"{h['teamCity']} {h['teamName']}".strip()

        if {"opponentTeamCity", "opponentTeamName"}.issubset(g.columns):
            away_team = f"{h.get('opponentTeamCity', '')} {h.get('opponentTeamName', '')}".strip()
        else:
            away_team = "Unknown"

        home_score = int(h["teamScore"])
        if "opponentScore" in g.columns and not pd.isna(h.get("opponentScore", np.nan)):
            away_score = int(h["opponentScore"])
        elif len(g) > 1:
            away_score = int(g.iloc[1]["teamScore"])
        else:
            away_score = 0

    games.append(
        {
            "gameId": game_id,
            "gameDate": game_date,
            "Home_Team": home_team,
            "Away_Team": away_team,
            "Home_Score": home_score,
            "Away_Score": away_score,
        }
    )

backtest_games = pd.DataFrame(games).sort_values("gameDate").reset_index(drop=True)

print("backtest_games shape:", backtest_games.shape)
print("Date range:", backtest_games["gameDate"].min(), "→", backtest_games["gameDate"].max())
display(backtest_games.head())


In [ ]:
from qepc.core.lambda_engine import LambdaConfig, compute_lambda_for_schedule

lam_cfg = LambdaConfig(
    home_advantage_points=2.5,
    min_lambda_total=190.0,
    max_lambda_total=260.0,
    use_net_rating_adjust=True,
    net_rating_weight=0.5,
)

# Use only the Home/Away columns for the λ builder
schedule_for_lambda = backtest_games[["Home_Team", "Away_Team"]].rename(
    columns={"Home_Team": "Home Team", "Away_Team": "Away Team"}
)

games_with_lambda_only = compute_lambda_for_schedule(
    schedule_df=schedule_for_lambda,
    strengths_df=strengths_df,
    config=lam_cfg,
    verbose=True,
)

# Merge λ back with actual scores
games_λ = backtest_games.join(
    games_with_lambda_only[["lambda_home", "lambda_away", "vol_home", "vol_away", "has_strengths"]]
)

print("games_λ shape:", games_λ.shape)
print("has_strengths value counts:")
print(games_λ["has_strengths"].value_counts(dropna=False))

display(games_λ.head())


In [ ]:
from qepc.core.simulator import SimConfig, run_qepc_simulation

# Keep only games where we have team strengths for both sides
valid_mask = games_λ["has_strengths"]
games_λ_valid = games_λ[valid_mask].reset_index(drop=True)

print("Games with strengths:", len(games_λ_valid), "out of", len(games_λ))

sim_cfg = SimConfig(
    num_trials=1000,    # trade-off: speed vs precision
    use_poisson=True,
    random_seed=42,
)

sim_results = run_qepc_simulation(
    games_df=games_λ_valid,
    config=sim_cfg,
    verbose=True,
)

print("sim_results shape:", sim_results.shape)
display(sim_results.head())


In [ ]:
sr = sim_results.copy()

# Actual outcomes
sr["Actual_Home_Win"] = sr["Home_Score"] > sr["Away_Score"]
sr["Actual_Total"] = sr["Home_Score"] + sr["Away_Score"]
sr["Actual_Spread"] = sr["Home_Score"] - sr["Away_Score"]

# QEPC predictions (using simulated means)
sr["Pred_Home_Win"] = sr["Home_Win_Prob"] > 0.5
sr["Pred_Total"] = sr["Sim_Home_Score"] + sr["Sim_Away_Score"]
sr["Pred_Spread"] = sr["Sim_Home_Score"] - sr["Sim_Away_Score"]

# Metrics
win_accuracy = (sr["Pred_Home_Win"] == sr["Actual_Home_Win"]).mean()
mae_total = (sr["Pred_Total"] - sr["Actual_Total"]).abs().mean()
mae_spread = (sr["Pred_Spread"] - sr["Actual_Spread"]).abs().mean()

print("📊 QEPC MULTI-SEASON BACKTEST")
print("Games evaluated:", len(sr))
print(f"Win accuracy:   {win_accuracy*100:.1f}%")
print(f"MAE total:      {mae_total:.2f} pts")
print(f"MAE spread:     {mae_spread:.2f} pts")


In [ ]:
sr["Error_Total"] = (sr["Pred_Total"] - sr["Actual_Total"]).abs()

print("🏆 Best 10 games by total error:")
display(
    sr.sort_values("Error_Total")
      .head(10)[["gameDate", "Home_Team", "Away_Team", "Pred_Total", "Actual_Total", "Error_Total"]]
)

print("\n⚠️ Worst 10 games by total error:")
display(
    sr.sort_values("Error_Total", ascending=False)
      .head(10)[["gameDate", "Home_Team", "Away_Team", "Pred_Total", "Actual_Total", "Error_Total"]]
)


# **Season Thus Far Backtest**

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

print("=== 2025-26 Season Backtest Setup ===")

team_stats_path = cfg.raw_root / "Team_Stats.csv"
print("Team_Stats path:", team_stats_path)

team_stats = pd.read_csv(team_stats_path)

# Parse dates robustly
team_stats["gameDate"] = pd.to_datetime(
    team_stats["gameDate"], errors="coerce", utc=True
).dt.tz_convert(None)

print("Raw Team_Stats shape:", team_stats.shape)
print("Date range:", team_stats["gameDate"].min(), "→", team_stats["gameDate"].max())
print("Columns:", list(team_stats.columns)[:20], "...")

display(team_stats.head())


In [ ]:
# Adjust these filters if your file uses different labels
season_filter = 2025
league_filter = "NBA"

ts = team_stats.copy()

if "season" in ts.columns:
    ts = ts[ts["season"] == season_filter]
if "league" in ts.columns:
    ts = ts[ts["league"] == league_filter]

ts = ts.dropna(subset=["gameId", "teamScore"])
ts["home"] = ts["home"].astype(float).round().astype(int)

print("Filtered Team_Stats for season/leauge:")
print("Shape:", ts.shape)
print("Date range:", ts["gameDate"].min(), "→", ts["gameDate"].max())
display(ts.head())

games_2025 = []

for game_id, g in ts.groupby("gameId"):
    g = g.sort_values("home")  # expect away=0, home=1

    game_date = g["gameDate"].iloc[0]

    home_row = g[g["home"] == 1]
    away_row = g[g["home"] == 0]

    if len(home_row) == 1 and len(away_row) == 1:
        h = home_row.iloc[0]
        a = away_row.iloc[0]

        home_team = f"{h['teamCity']} {h['teamName']}".strip()
        away_team = f"{a['teamCity']} {a['teamName']}".strip()

        home_score = int(h["teamScore"])
        away_score = int(a["teamScore"])
    else:
        # Fallback for weird single-row games (should be rare)
        h = g.iloc[0]
        home_team = f"{h['teamCity']} {h['teamName']}".strip()

        if {"opponentTeamCity", "opponentTeamName"}.issubset(g.columns):
            away_team = f"{h.get('opponentTeamCity', '')} {h.get('opponentTeamName', '')}".strip()
        else:
            away_team = "Unknown"

        home_score = int(h["teamScore"])
        if "opponentScore" in g.columns and not pd.isna(h.get("opponentScore", np.nan)):
            away_score = int(h["opponentScore"])
        elif len(g) > 1:
            away_score = int(g.iloc[1]["teamScore"])
        else:
            away_score = 0

    games_2025.append(
        {
            "gameId": game_id,
            "gameDate": game_date,
            "Home_Team": home_team,
            "Away_Team": away_team,
            "Home_Score": home_score,
            "Away_Score": away_score,
        }
    )

season_2025_games = (
    pd.DataFrame(games_2025)
    .sort_values("gameDate")
    .reset_index(drop=True)
)

print("\n=== 2025-26 Season Game Table ===")
print("Games:", len(season_2025_games))
print("Date range:", season_2025_games["gameDate"].min(), "→", season_2025_games["gameDate"].max())
display(season_2025_games.head(10))
